# IMPORTS

In [3]:
import cv2
import numpy as np
import os

# 1. Function for the Preprocessing of Image
- Grayscale and Bilateral Filter

In [2]:
def preprocess_image(image):
    """Converts to grayscale and applies a bilateral filter."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17) 
    return bfilter

# 2. Function for the Image Pyramid

In [ ]:
# this is from the working sample code
def pyramid(image, scale=1.5, minSize=(30, 30)):
    """Yields images at different scales (an image pyramid) using cv2.resize"""
    # yield the original image
    yield image
    
    while True:
        # compute the new width
        w = int(image.shape[1] / scale)
        
        # compute the new height, maintaining aspect ratio
        r = w / float(image.shape[1])
        h = int(image.shape[0] * r)
        
        # new dimensions
        dim = (w, h)
        
        # resize the image using cv2 (INTER_AREA is good for shrinking)
        image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
        
        # if the resized image is too small, stop
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
            
        # yield the next image in the pyramid
        yield image

In [ ]:
# this is from the HOA 11.1
def image_pyramid(image, scale=0.7, min_size=500):
    img = image.copy()
    level = 0
    while img.shape[0] > min_size and img.shape[1] > min_size:
        img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
        img = cv2.GaussianBlur(img, (5, 5), 0)
        print(f"Pyramid Level {level}: shape={img.shape}")
        level += 1

# 3. Finding the License Plate Contours

In [ ]:
def find_plate_contour(processed_image):
    """Finds the 4-point contour of the license plate."""
    try:
        edged = cv2.Canny(processed_image, 30, 200) 
        contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

        plate_contour = None
        for c in contours:
            perimeter = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.018 * perimeter, True)

            if len(approx) == 4:
                (x, y, w, h) = cv2.boundingRect(approx)
                aspect_ratio = float(w) / h
                
                # Tune this aspect ratio range
                if 2.0 < aspect_ratio < 4.5 and w > 30 and h > 15:
                    plate_contour = approx
                    break
        
        return plate_contour
    except Exception as e:
        print(f"Error in find_plate_contour: {e}")
        return None

# 4. Cropping the Image to Isolate the License Plate

In [ ]:
def crop_plate(image, plate_contour, plate_type="car"):
    """Crops and warps the 4-point contour to a flat, top-down image. Supports car and motorcycle plate sizes."""
    try:
        pts = plate_contour.reshape(4, 2)
        rect = np.zeros((4, 2), dtype="float32")

        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)] # Top-left
        rect[2] = pts[np.argmax(s)] # Bottom-right

        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)] # Top-right
        rect[3] = pts[np.argmax(diff)] # Bottom-left

        """
        For car plates, use 390x140
        For motorcycle plates, use 235x135
        """        
        if plate_type == "motorcycle":
            target_w = 235
            target_h = 135
        else:  # default to car
            target_w = 390
            target_h = 140

        dst = np.array([
            [0, 0],
            [target_w - 1, 0],
            [target_w - 1, target_h - 1],
            [0, target_h - 1]], dtype="float32")

        M = cv2.getPerspectiveTransform(rect, dst)
        cropped = cv2.warpPerspective(image, M, (target_w, target_h))

        cropped_gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
        return cropped_gray
    except Exception as e:
        print(f"Error in crop_plate: {e}")
        return None

# 5. Load the Template of the Characters for Matching

In [ ]:
def load_templates(template_directory="templates"):
    """Loads all templates from a directory into a dictionary."""
    templates = {}
    if not os.path.exists(template_directory):
        print(f"Error: Template directory not found at {template_directory}")
        return None

    print(f"Loading templates from {template_directory}...")
    for filename in os.listdir(template_directory):
        if filename.endswith(('.png', '.jpg', '.bmp')):
            char_name = os.path.splitext(filename)[0]
            template_img = cv2.imread(os.path.join(template_directory, filename), 
                                      cv2.IMREAD_GRAYSCALE)
            
            if template_img is None: continue
            # Use THRESH_BINARY to match the segmented characters (white on black)
            _, template_thresh = cv2.threshold(template_img, 127, 255, cv2.THRESH_BINARY)
            
            # CRITICAL: Tightly crop the template to remove padding/whitespace
            coords = cv2.findNonZero(template_thresh)
            if coords is not None:
                x, y, w, h = cv2.boundingRect(coords)
                template_thresh = template_thresh[y:y+h, x:x+w]
            
            templates[char_name] = template_thresh
    
    return templates

# 6. Segment the Characters in the Cropped License Plate